# Testing the deductron layer

This notebook implements one of the examples from the Keras documentation for sequence-to-sequence learning using RNNs. The task is to solve addition problems, formulated as strings. For example, the network should be able to take the string `341+78 ` and return the sum `419`.

This is accomplished using an encoding layer and a decoding layer. In the documentation example, both the encoder and the decoder are LSTMs. In this notebook, we generate some training data and train networks using both LSTMs and deductron layers.

The code for this example is taken from https://keras.io/examples/nlp/addition_rnn/ and lightly modified to fit our use case.

In [1]:
import keras
from keras import layers
import numpy as np
import deductron

# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True        # Reverse the order of the input strings -- this seems to improve learning

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# Although a sigmoid is a natural choice for an activation with values in (0, 1), a ReLU
# constrained to produce output in [0, 1] has much better performance.
clipped_relu = lambda x: keras.activations.relu(x, max_value=1)

Using TensorFlow backend.


In [2]:
# Generate addition problems for training

class CharacterTable:
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """

    def __init__(self, chars):
        """Initialize character table.
        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.
        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.
        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[x] for x in x)


# All the numbers, plus sign and space for padding.
chars = "0123456789+ "
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
while len(questions) < TRAINING_SIZE:
    f = lambda: int(
        "".join(
            np.random.choice(list("0123456789"))
            for i in range(np.random.randint(1, DIGITS + 1))
        )
    )
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = "{}+{}".format(a, b)
    query = q + " " * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += " " * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)

In [3]:
# Vectorize the training and validation data

x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

## Reference example: LSTM encoder/decoder

The purpose of the next few sections is to compare the LSTM solution with a pure deductron solution as well as mixed LSTM-deductron solutions.

The reference approach uses two layers, thinking of the first as an encoder and the second as a decoder. The first layer extracts features from the input and produces an internal representation of the problem. The final output of this "encoder" layer is then repeated to form a constant sequence and passed to the "decoder" layer as its input.

Following the existing example, we begin by building a model in which each of the encoder and decoder is a single LSTM layer with 128 hidden units. Further examples will replace each of these layers in turn with a deductron layer (also with 128 units).

In [4]:
model = keras.Sequential()
# "Encode" the input sequence using a LSTM, producing an output of size 128.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
# Replacing the encoder with a deductron layer seems to hurt accuracy.
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# We'll use a single layer for the decoder -- adding more layers doesn't seem to improve results
model.add(layers.LSTM(128, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
dense_1 (Dense)              (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


Now it is time to train the model. We will train all of the models for 12 epochs (more would be better, but the training is a little time consuming). After each epoch we visualize 10 samples from the validation set so that we can see the sort of output we are getting.

In [5]:
epochs = 12
batch_size = 32

# Train the model each generation and show predictions against the validation
# dataset.
for epoch in range(0, epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


Iteration 0
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 17s 378us/step - loss: 1.7643 - accuracy: 0.3549 - val_loss: 1.5724 - val_accuracy: 0.4114
Q 450+30  T 480  ☒ 555 
Q 53+854  T 907  ☒ 603 
Q 601+43  T 644  ☒ 657 
Q 280+21  T 301  ☒ 235 
Q 619+996 T 1615 ☒ 1633
Q 891+321 T 1212 ☒ 1035
Q 43+545  T 588  ☒ 555 
Q 249+158 T 407  ☒ 587 
Q 63+639  T 702  ☒ 687 
Q 823+491 T 1314 ☒ 1135

Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 16s 359us/step - loss: 1.3714 - accuracy: 0.4862 - val_loss: 1.1968 - val_accuracy: 0.5591
Q 86+76   T 162  ☒ 155 
Q 40+292  T 332  ☒ 319 
Q 43+16   T 59   ☒ 40  
Q 760+2   T 762  ☒ 771 
Q 608+56  T 664  ☒ 689 
Q 653+984 T 1637 ☒ 1509
Q 486+409 T 895  ☒ 809 
Q 33+39   T 72   ☒ 40  
Q 750+73  T 823  ☒ 814 
Q 998+90  T 1088 ☒ 1023

Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===============

Typically, after 12 epochs, the above network should reach about 98-99% accuracy, with most visualized problems solved correctly.

## A pure deductron implementation

Our first change is to replace both the encoder and decoder LSTM layers with deductron layers.

In [1]:
model = keras.Sequential()
model.add(deductron.Deductron(256, activation = clipped_relu, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))
model.add(deductron.Deductron(256, activation = clipped_relu, return_sequences=True))

model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

NameError: name 'keras' is not defined

In [ ]:
for epoch in range(epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)


Iteration 0
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
 8864/45000 [====>.........................] - ETA: 10s - loss: 2.0272 - accuracy: 0.2970

The pure deductron network does improve accuracy with training, but learns much more slowly, reaching only around 58-60% accuracy after 12 epohcs. Visual inspection of the output shows many repeated digits and similar apparent patterns, possibly pointing to insufficient feature extraction.

## Mixed solutions

In [ ]:
# LSTM encoder, deductron decoder
model = keras.Sequential()
model.add(layers.LSTM(128, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))

model.add(deductron.Deductron(128, activation = clipped_relu, return_sequences=True))

model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

In [ ]:
for epoch in range(epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)

Accuracy with the LSTM encoder and deductron decoder is comparable to using LSTM in both roles. This adds credibility to the idea that the problem in the previous test was insufficient feature extraction by the deductron layer.

Reversing the roles of the LSTM and deductron adds more weight to this idea.

In [ ]:
# Deductron encoder, LSTM decoder
num_layers = 1  # Number of layers in the decoder.
model = keras.Sequential()
model.add(deductron.Deductron(128, activation = clipped_relu, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))

model.add(layers.LSTM(128, return_sequences=True))

model.add(layers.Dense(len(chars), activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

In [ ]:
for epoch in range(epochs):
    print()
    print("Iteration", epoch)
    model.fit(
        x_train,
        y_train,
        batch_size=batch_size,
        epochs=1,
        validation_data=(x_val, y_val),
    )
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = np.argmax(model.predict(rowx), axis=-1)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print("Q", q[::-1] if REVERSE else q, end=" ")
        print("T", correct, end=" ")
        if correct == guess:
            print("☑ " + guess)
        else:
            print("☒ " + guess)

This network has only marginally better performance than the pure deductron, suggesting that the low performance was tied specifically to the deductron in the encoder role, not the absence of an LSTM layer in the pure deductron network.

## Discussion

The above tests show that the deductron architecture is capable of learning the sequence-to-sequence addition task. However, in the encoder-decoder network structure, the deductron does not perform as well as LSTM in the "encoder" role. Using LSTM for the encoder and deductron for the decoder reaches approximate parity with the LSTM-LSTM network; reversing the roles leads to slower learning. This suggests that the deductron may not have enough complexity to extract the necessary features for this task, but is effective at using these features once learned by the LSTM layer.